# 1. Basic recommendations

## 1.1 Reading ratings

In [1]:
import pandas as pd
import datetime

class UserItemData:
    
    read_counter = 0
    
    def __init__(self, path, from_date = None, to_date = None, min_ratings = None):
        self.path = path
        self.from_date = from_date
        self.to_date = to_date
        self.min_ratings = min_ratings
        self.data = pd.read_table(self.path)
        
        
        
        
        if self.from_date != None and self.to_date != None:
            from_date_dt = datetime.datetime.strptime(from_date, "%d.%m.%Y")
            to_date_dt = datetime.datetime.strptime(to_date, "%d.%m.%Y")
#             print('From: ' +str(from_date_dt)+ '\tTo: ' +str(to_date_dt))
            
            self.data['date_combined'] = pd.to_datetime(dict(year = self.data.date_year, month = self.data.date_month, day = self.data.date_day))
            # тут, где >= from_date_dt, где < to_date_dt и > self.min_ratings - это я догадывался, что включать, а что не,
            # чтобы мой аутпут сходился с ее аутпутом
            self.data = self.data[(self.data.date_combined >= from_date_dt) & (self.data.date_combined < to_date_dt)]
            
        if self.from_date != None and self.to_date == None:
            
            from_date_dt = datetime.datetime.strptime(from_date, "%d.%m.%Y")
            
            self.data['date_combined'] = pd.to_datetime(dict(year = self.data.date_year, month = self.data.date_month, day = self.data.date_day))
            self.data = self.data[(self.data.date_combined >= from_date_dt)]
        
        if self.from_date == None and self.to_date != None:
            
            to_date_dt = datetime.datetime.strptime(to_date, "%d.%m.%Y")
            
            self.data['date_combined'] = pd.to_datetime(dict(year = self.data.date_year, month = self.data.date_month, day = self.data.date_day))
            self.data = self.data[(self.data.date_combined < to_date_dt)]
        
        
        
        
        
        if self.min_ratings != None:
            self.data = self.data.groupby('movieID').filter(lambda s: s.rating.count() > self.min_ratings)
        
        
        
        
        
    def print_data(self):
        display(self.data)
        
    def nratings(self):
        read_counter = len(self.data.index)
#         print('Program has read ' +str(read_counter)+ ' rows.')
        return read_counter
    
    def save(self, save_path):
        self.data.to_csv(save_path, sep = '\t', index=False)
        

test = UserItemData('data/user_ratedmovies.dat')
# test.print_data()
print(test.nratings())

test2 = UserItemData('data/user_ratedmovies.dat', min_ratings = 100, from_date = '12.1.2007', to_date = '16.2.2008')
# test2.print_data()
print(test2.nratings())
# save_path = 'output/output1.dat'
# test2.save(save_path)

# test3 = UserItemData(save_path)
# test3.print_data()
# print(test3.nratings())
# test3.save('output/output2.dat')

855598
72784


## 1.2 Reading movies

In [2]:
class MovieData:
    def __init__(self, path):
        self.path = path
        self.data = pd.read_table(self.path, encoding='latin-1')
    
    def get_title(self, movieID):
        return self.data.loc[self.data['id'] == movieID, 'title'].iloc[0]

md = MovieData('data/movies.dat')
print(md.get_title(1))

Toy story


## 1.3 Random predictor

In [3]:
import random

class RandomPredictor:
    def __init__(self, min_rat, max_rat):
        self.min_rat = min_rat
        self.max_rat = max_rat
        
    def fit(self, X):
        self.X = X
        
    def predict(self, user_id): # note that user_id is used nowhere here at all as it has no influence here because it is just a random prediction...
        recommend_dict = {}
        for i in self.X.data.movieID.unique(): # iterate though the list of unique movieID-s that appear in this particular data
            recommend_dict[i] = random.randint(self.min_rat, self.max_rat)
        return recommend_dict

    
    
md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = RandomPredictor(1, 5)
rp.fit(uim)
pred = rp.predict(78)
print(type(pred))
items = [1, 3, 20, 50, 100]
for item in items:
    print("Movie: {}, score: {}".format(md.get_title(item), pred[item]))

<class 'dict'>
Movie: Toy story, score: 3
Movie: Grumpy Old Men, score: 5
Movie: Money Train, score: 3
Movie: The Usual Suspects, score: 2
Movie: City Hall, score: 3


## 1.4 Recommendation

In [4]:
class Recommender:
    
    def __init__(self, predictor):
        self.predictor = predictor
    
    def fit(self, X):
        self.X = X
        self.predictor.fit(X)
    
    def recommend(self, userID, n = 10, rec_seen = True):
        
        pred = self.predictor.predict(userID)
        pred_list = list(pred.items()) # we need to sort this, so dictionary will not help us here, so convert this to the list of tupples
        pred_list_sorted = sorted(pred_list, key=lambda x: x[1], reverse = True) # sort from rating 5 to rating 1
        recommend_list = []
        
        
        if rec_seen == False:
            table_seen = self.X.data[self.X.data.userID == userID]
            column_seen = table_seen['movieID'].tolist()
            
            tup_dict = dict(pred_list_sorted) # in order to reach first element of each tupple, we convert this back to the dictionary
            for i in column_seen:
                tup_dict.pop(i) # remove already seen movie
            # чувак их инета сказал, что перед тапл надо поставить лист. Я чекнул, но разницы не заметил, но все равно оставлю лист на всякий:
            pred_list_sorted = list(tuple(tup_dict.items())) # and then in the end again convert it to the list of tupples
        
        
        for i in range(n):
            recommend_list.append(pred_list_sorted[i])
        
#         print(recommend_list)
        
        return recommend_list
    



md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = RandomPredictor(1, 5)
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Movie: {}, score: {}".format(md.get_title(idmovie), val))

Movie: Congo, score: 5
Movie: Die Hard, score: 5
Movie: Armageddon, score: 5
Movie: Superman, score: 5
Movie: Hart's War, score: 5


## 1.5 Average predictor

In [5]:
import numpy as np

class AveragePredictor:
    
    def __init__(self, b):
        self.b = b
        
    def fit(self, X):
        self.X = X
    
    def avg_calc(self, n, vs):
        avg = (vs + self.b * self.g_avg) / (n + self.b)
        return avg
        
    def predict(self, user_id): # note that user_id is used nowhere here at all as it has no influence here because it is just a random prediction...
        
        recommend_dict = {}
        
        self.g_avg = self.X.data.rating.mean()
        
        # first, create new table with movieID and n cols:
        new_table = self.X.data.groupby(['movieID'])['rating'].count().reset_index(name="n")
        # secondly, create another table (consisting of one col only) for another function"
        vs_column = self.X.data.groupby(['movieID'])['rating'].sum().reset_index(name="vs")['vs']
        # add previous column to the previous table:
        new_table['vs'] = vs_column
#         display(new_table)
        
        new_table['avg'] = np.vectorize(self.avg_calc)(new_table['n'], new_table['vs'])
        
        for i in new_table.movieID.unique(): # iterate though the list of unique movieID-s that appear in this particular data
                                                # вообще-то тут нету смысла в unique, ибо new_table и так уже содержит unique movieID
            
            recommend_dict[i] = new_table.loc[new_table.movieID == i, 'avg'].iloc[0]
        
        
        return recommend_dict


md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
print('Run b = 0 :')
rp = AveragePredictor(b=0)
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Movie: {}, score: {}".format(md.get_title(idmovie), val))

print('Run b = 100 :')
rp = AveragePredictor(b=100)
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Movie: {}, score: {}".format(md.get_title(idmovie), val))

Run b = 0 :
Movie: Brother Minister: The Assassination of Malcolm X, score: 5.0
Movie: Synthetic Pleasures, score: 5.0
Movie: Gabbeh, score: 5.0
Movie: Storefront Hitchcock, score: 5.0
Movie: Ko to tamo peva, score: 5.0
Run b = 100 :
Movie: The Usual Suspects, score: 4.225944245560473
Movie: The Godfather: Part II, score: 4.146907937910189
Movie: Cidade de Deus, score: 4.116538340205236
Movie: The Dark Knight, score: 4.10413904093503
Movie: 12 Angry Men, score: 4.103639627096175


## 1.6 Recommending the most watched movies

In [6]:
class ViewsPredictor:
        
    def fit(self, X):
        self.X = X
        
    def predict(self, user_id): # note that user_id is used nowhere here at all as it has no influence here because it is just a random prediction...
        
        recommend_dict = {}
        
        # тут я специально создал новую переменную, ибо если буду перезаписывать в старую self.X.data,
        # то потом уже после этого класса, другая часть кода будет опираться на МОДИФИЦИРОВАННЫЙ self.X.data,
        # поэтому тут создаем свою переменную, где будут только 2 + 1 = 3 колонки
        grouped_data = self.X.data.groupby(['movieID'])['movieID'].count().reset_index(name="movieID_count")
        
#         display(grouped_data)
            
        recommend_dict = grouped_data.set_index('movieID').to_dict()['movieID_count']
        
        return recommend_dict

md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat')
rp = ViewsPredictor()
rec = Recommender(rp)
rec.fit(uim)
rec_items = rec.recommend(78, n=5, rec_seen=False)
for idmovie, val in rec_items:
    print("Movie: {}, score: {}".format(md.get_title(idmovie), val))

Movie: The Lord of the Rings: The Fellowship of the Ring, score: 1576
Movie: The Lord of the Rings: The Two Towers, score: 1528
Movie: The Lord of the Rings: The Return of the King, score: 1457
Movie: The Silence of the Lambs, score: 1431
Movie: Shrek, score: 1404


# 2. Collaborative filtering

## 2.1 Predicting scores with similarity between products

In [7]:
import numpy as np
import time

class ItemBasedPredictor:

    def __init__(self, min_values = 0, threshold = 0):
        self.min_values = min_values
        self.threshold = threshold
    
    def fit(self, X):
        
        self.X = X
        
        self.similarities = pd.DataFrame(index = self.X.data.movieID.unique())
        for i in self.similarities.index:
            self.similarities[str(i)] = np.nan

        
        
        
    
    def find_avgRating(self, userID):
        one_movie_rating = self.X.data[self.X.data.userID == userID]['rating'].mean()
        return one_movie_rating
    
    def getRatingDifference(self, userID, avgRating, p):
        rating = self.X.data.loc[((self.X.data.userID == userID) & (self.X.data.movieID == p)), 'rating'].iloc[0]
        return rating - avgRating
    
    def cols_product(self, col1, col2):
        return col1 * col2
    
    
    
    
    
    def predict(self, userID): # а вот теперь мы этот userID наконец-то юзаем, гы
        
        start_time_prediction = time.time()
        recommend_dict = {}
        
        unique_movies = self.X.data.movieID.unique()
#         display(unique_movies)
        
        for i in unique_movies: # iterate though the list of unique movieID-s that appear in this particular data
            
            start_time_movie = time.time()
            
            personal = self.X.data[self.X.data.userID == userID][['movieID', 'rating']]
            personal['similarity'] = np.nan

            remove_index = personal[personal.movieID == i].index
#             print('Next index to remove: ' +str(remove_index))
            personal = personal.drop(remove_index)

            personal['similarity'] = np.vectorize(self.similarity)(i, personal['movieID'])
            personal['product'] = np.vectorize(self.cols_product)(personal['rating'], personal['similarity'])
            
#             display(personal)
            numerator = personal['product'].sum()
            denominator = personal['similarity'].sum()

            result = numerator / denominator
#             print('The recommentation for movieID ' +str(i)+ ' is: ' +str(result))
            
            recommend_dict[i] = result
            
            end_time_movie = time.time()
            time_movie = end_time_movie - start_time_movie
            print('Time needed for movieID ' +str(i)+ ': ' +str(time_movie)+ ' seconds.')
        
        end_time_prediction = time.time()
        time_prediction = end_time_prediction - start_time_prediction
        print('Time spent for the whole RECOMMENDATION: ' +str(time_prediction)+ ' seconds.')
        
        # Save similarities to use it in the future...
        # Index must be true here so that we can further read the firct column as index column:
        self.similarities.to_csv('output/calculated_similarities.dat', sep = '\t', index=True)
        
        return recommend_dict
    
    
    
    
    
    def similarity(self, p1, p2):
        
        # Check if this is NaN value:
        if pd.isna(self.similarities.at[p1, str(p2)]) == True:
            if p1 != p2:
                sim_res = self.similarity_calculate(p1, p2)
                self.similarities.at[p1, str(p2)] = sim_res
                self.similarities.at[p2, str(p1)] = sim_res
            else:
                return np.nan
        
        return self.similarities.at[p1, str(p2)]
    
    
    def similarity_calculate(self, p1, p2):
        
        data = self.X.data
        
        # Get all users that have rated p1 or p2:
        relevant_data = data[(data['movieID'] == p1) | (data['movieID'] == p2) ][['userID', 'movieID']]
        # Get only those users that have rated BOTH p1 and p2 movies:
        relevant_data = relevant_data[relevant_data.groupby(['userID'])['userID'].transform('count') == 2]
        # Get unuqie column (table) of users:
        relevant_data = relevant_data[['userID']].drop_duplicates()
        
        table = pd.DataFrame(columns = ['userID', 'avgRating', 'movie1_diff', 'movie2_diff'])
        table['userID'] = relevant_data['userID']
        table = table.reset_index(drop = True)
        
        if len(table.index) < self.min_values:
#             print('min_values applied:')
            return float(0)
        
        table['avgRating'] = np.vectorize(self.find_avgRating)(table['userID'])
        table['movie1_diff'] = np.vectorize(self.getRatingDifference)(table['userID'], table['avgRating'], p1)
        table['movie2_diff'] = np.vectorize(self.getRatingDifference)(table['userID'], table['avgRating'], p2)
        table['diffs_product'] = np.vectorize(self.cols_product)(table['movie1_diff'], table['movie2_diff'])
        
        numenator = table['diffs_product'].sum()
        
        table['part1'] = np.vectorize(self.cols_product)(table['movie1_diff'], table['movie1_diff'])
        part1 = table['part1'].sum()
        part1 = part1 ** 0.5 # square root
        
        table['part2'] = np.vectorize(self.cols_product)(table['movie2_diff'], table['movie2_diff'])
        part2 = table['part2'].sum()
        part2 = part2 ** 0.5 # square root
        
        denominator = part1 * part2
        result = numenator / denominator
        
        if result < self.threshold:
#             print('threshold applied:')
            result = 0
    
        return float(result)
    
    
    
    
    
    def similarItems(self, item, n):
        
        movie_column = pd.DataFrame(columns = [str(item)])
        movie_column[str(item)] = self.similarities[str(item)]
#         display(movie_column)
        
        simil_dict = movie_column.to_dict()
#         print(simil_dict)
                    
        simil_list = list(simil_dict[str(item)].items())
        
        simil_list_sorted = sorted(simil_list, key = lambda x: x[1], reverse = True) # sort from biggest similarity to the lowest
        simil_list_sorted = [(x, y) for x, y in simil_list_sorted if pd.isna(y) == False] # remove NaN values
        simil_list_sorted_the_best = simil_list_sorted[0:n] # note that we include a pointer to the next value, which is n (the last value in our list will be with index n-1, although here we see n)
        
        return simil_list_sorted_the_best
    
    
    
    
    
    


md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat', min_ratings=1000)
rp = ItemBasedPredictor()
rec = Recommender(rp)
rec.fit(uim)

print('Calculating similarities: ')
print("Similarity between the movies 'Men in black'(1580) and 'Ghostbusters'(2716): ", rp.similarity(1580, 2716))
print("Similarity between the movies 'Men in black'(1580) and 'Schindler's List'(527): ", rp.similarity(1580, 527))
print("Similarity between the movies 'Men in black'(1580) and 'Independence day'(780): ", rp.similarity(1580, 780))

print('Now, we will NOT calculate it, but simply GET in from the table:')
print("Similarity between the movies 'Men in black'(1580) and 'Ghostbusters'(2716): ", rp.similarity(1580, 2716))
print("Similarity between the movies 'Men in black'(1580) and 'Schindler's List'(527): ", rp.similarity(1580, 527))
print("Similarity between the movies 'Men in black'(1580) and 'Independence day'(780): ", rp.similarity(1580, 780))

print('Checking vice-versa coordinates:')
print("Similarity between the movies 'Men in black'(1580) and 'Ghostbusters'(2716): ", rp.similarity(2716, 1580))
print("Similarity between the movies 'Men in black'(1580) and 'Schindler's List'(527): ", rp.similarity(527, 1580))
print("Similarity between the movies 'Men in black'(1580) and 'Independence day'(780): ", rp.similarity(780, 1580))


# New run:

md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat', min_ratings=1000)
rp = ItemBasedPredictor()
rec = Recommender(rp)
rec.fit(uim)

# In case you do not want to wait for the calculations, you can UNcomment these two lines
# and you would simply read the calculations that were done BEFORE:
# rp.similarities = pd.read_table('output/calculated_similarities.dat', index_col = 0)
# display(rp.similarities)

print("Predictions for 78: ")
rec_items = rec.recommend(78, n=15, rec_seen=False)
for idmovie, val in rec_items:
    print("Movie: {}, score: {}".format(md.get_title(idmovie), val))

Calculating similarities: 
Similarity between the movies 'Men in black'(1580) and 'Ghostbusters'(2716):  0.23395523176756633
Similarity between the movies 'Men in black'(1580) and 'Schindler's List'(527):  0.0
Similarity between the movies 'Men in black'(1580) and 'Independence day'(780):  0.42466125844687613
Now, we will NOT calculate it, but simply GET in from the table:
Similarity between the movies 'Men in black'(1580) and 'Ghostbusters'(2716):  0.23395523176756633
Similarity between the movies 'Men in black'(1580) and 'Schindler's List'(527):  0.0
Similarity between the movies 'Men in black'(1580) and 'Independence day'(780):  0.42466125844687613
Checking vice-versa coordinates:
Similarity between the movies 'Men in black'(1580) and 'Ghostbusters'(2716):  0.23395523176756633
Similarity between the movies 'Men in black'(1580) and 'Schindler's List'(527):  0.0
Similarity between the movies 'Men in black'(1580) and 'Independence day'(780):  0.42466125844687613
Predictions for 78: 
Ti

In [8]:
# display(rp.similarities)
# # That's how we should save the similarities so that they can help us in the future:
# rp.similarities.to_csv('output/calculated_similarities_copy.dat', sep = '\t', index=True)
# # That's how we should read them:
# saved_similarities = pd.read_table('output/calculated_similarities_copy.dat', index_col = 0)
# display(saved_similarities)

## 2.2 Most similar movies

In [9]:
# Let's calculate all remaining similarities:

# In case you do not want to wait for the calculations, you can UNcomment these two lines
# and you would simply read the calculations that were done BEFORE:
# rp.similarities = pd.read_table('output/all_calculated_similarities.dat', index_col = 0)
# display(rp.similarities)

start = time.time()

for column in rp.similarities: # iterate through every column to calculate remaining similarities:
    rp.similarities[column] = np.vectorize(rp.similarity)(rp.similarities.index, int(column))

end = time.time()
time_spent = end - start
# print('Time needed for remaining calculations of similarities: ' +str(time_spent))

rp.similarities.to_csv('output/all_calculated_similarities.dat', sep = '\t', index=True)






# Now, let's play with similarities when we have all of them:

simil_dict = rp.similarities.to_dict()
simil_list = []

for k1, v1 in simil_dict.items():
    for k2, v2 in v1.items():
        if pd.isna(v2) == False:
            # int(k1) because it is column, which means it's a string, but we want integer for simplicity:
            local_tuple = (int(k1), k2, v2)
            simil_list.append(local_tuple)

simil_list_sorted = sorted(simil_list, key=lambda x: x[2], reverse = True) # sort from biggest similarity to the lowest

for i in range(20):
    print('Movie 1: ' +md.get_title(simil_list_sorted[i][0])+ ', Movie2: ' +md.get_title(simil_list_sorted[i][1])+ ', Similarity: ' +str(simil_list_sorted[i][2]))

Movie 1: The Lord of the Rings: The Two Towers, Movie2: The Lord of the Rings: The Return of the King, Similarity: 0.8439842148481417
Movie 1: The Lord of the Rings: The Return of the King, Movie2: The Lord of the Rings: The Two Towers, Similarity: 0.8439842148481417
Movie 1: The Lord of the Rings: The Fellowship of the Ring, Movie2: The Lord of the Rings: The Two Towers, Similarity: 0.8231885401761888
Movie 1: The Lord of the Rings: The Two Towers, Movie2: The Lord of the Rings: The Fellowship of the Ring, Similarity: 0.8231885401761888
Movie 1: The Lord of the Rings: The Fellowship of the Ring, Movie2: The Lord of the Rings: The Return of the King, Similarity: 0.8079374897442496
Movie 1: The Lord of the Rings: The Return of the King, Movie2: The Lord of the Rings: The Fellowship of the Ring, Similarity: 0.8079374897442496
Movie 1: Kill Bill: Vol. 2, Movie2: Kill Bill: Vol. 2, Similarity: 0.7372340224381029
Movie 1: Kill Bill: Vol. 2, Movie2: Kill Bill: Vol. 2, Similarity: 0.737234022

## 2.3 Recommendation based on the currently viewed content

In [10]:
rec_items = rp.similarItems(4993, 10)
print('Movies similar to "The Lord of the Rings: The Fellowship of the Ring": ')
for idmovie, val in rec_items:
    print("Movie: {}, score: {}".format(md.get_title(idmovie), val))

Movies similar to "The Lord of the Rings: The Fellowship of the Ring": 
Movie: The Lord of the Rings: The Two Towers, score: 0.8231885401761888
Movie: The Lord of the Rings: The Return of the King, score: 0.8079374897442496
Movie: Star Wars: Episode V - The Empire Strikes Back, score: 0.2396194307349645
Movie: Star Wars, score: 0.2196558652707407
Movie: The Matrix, score: 0.2151555270688023
Movie: Raiders of the Lost Ark, score: 0.19944276706345015
Movie: The Usual Suspects, score: 0.18321188451910753
Movie: Blade Runner, score: 0.16399681315410275
Movie: Schindler's List, score: 0.16105905138148702
Movie: Monty Python and the Holy Grail, score: 0.15780453798519137


## 2.4 Recommendation for yourself

In [11]:
md_1 = MovieData('data/movies.dat')
uim_1 = UserItemData('data/user_ratedmovies_additional.dat', min_ratings=1000)
rp_1 = ItemBasedPredictor()
rec_1 = Recommender(rp_1)
rec_1.fit(uim_1)

# In case you do not want to wait for the calculations, you can UNcomment these two lines
# and you would simply read the calculations that were done BEFORE:
# rp_1.similarities = pd.read_table('output/all_calculated_similarities.dat', index_col = 0)
# display(rp_1.similarities)

print("Predictions for 78: ")
rec_items_1 = rec_1.recommend(888888, n=15, rec_seen=False)
for idmovie, val in rec_items_1:
    print("Movie: {}, score: {}".format(md.get_title(idmovie), val))

Predictions for 78: 
Time needed for movieID 32: 32.953595876693726 seconds.
Time needed for movieID 110: 34.130043745040894 seconds.
Time needed for movieID 296: 42.17946410179138 seconds.
Time needed for movieID 589: 38.13544940948486 seconds.
Time needed for movieID 1036: 30.97529911994934 seconds.
Time needed for movieID 1527: 33.079095125198364 seconds.
Time needed for movieID 2571: 41.14597487449646 seconds.
Time needed for movieID 2762: 42.21225547790527 seconds.
Time needed for movieID 2959: 31.552284002304077 seconds.
Time needed for movieID 3793: 31.594585418701172 seconds.
Time needed for movieID 4993: 29.439985036849976 seconds.
Time needed for movieID 5952: 24.118951082229614 seconds.
Time needed for movieID 7153: 19.319316625595093 seconds.
Time needed for movieID 32587: 29.230201959609985 seconds.
Time needed for movieID 150: 28.77849817276001 seconds.
Time needed for movieID 260: 16.42332935333252 seconds.
Time needed for movieID 318: 11.893150568008423 seconds.
Time ne

## 2.5 Prediction with Slope One method

In [14]:
class SlopeOnePredictor:
    
    def fit(self, X):
        
        self.X = X
        
        self.deviations = pd.DataFrame(index = self.X.data.movieID.unique())
        for i in self.deviations.index:
            self.deviations[str(i)] = np.nan
        
        self.deviations_count = pd.DataFrame(index = self.X.data.movieID.unique())
        for i in self.deviations_count.index:
            self.deviations_count[str(i)] = np.nan

        
    
    
    
    def get_deviation_denominator(self, p1, p2):
        return self.deviations_count.at[p1, str(p2)]
    
    def calc_movies_diff(self, userID, p1, p2):
        
        movie_1 = self.X.data.loc[((self.X.data.userID == userID) & (self.X.data.movieID == p1)), 'rating'].iloc[0]
        movie_2 = self.X.data.loc[((self.X.data.userID == userID) & (self.X.data.movieID == p2)), 'rating'].iloc[0]
        return movie_1 - movie_2
    
    
    def numerator_part(self, col1, col2, userID, movieID):
        rating = self.X.data.loc[((self.X.data.userID == userID) & (self.X.data.movieID == movieID)), 'rating'].iloc[0]
        return (col1 + rating) * col2
    
    
    
    
    
    def predict(self, userID): # а вот теперь мы этот userID наконец-то юзаем, гы
        
        start_time_prediction = time.time()
        recommend_dict = {}
        
        unique_movies = self.X.data.movieID.unique()
#         display(unique_movies)
        
        for i in unique_movies: # iterate though the list of unique movieID-s that appear in this particular data
            
            start_time_movie = time.time()
            
            personal = self.X.data[self.X.data.userID == userID][['movieID']]
            personal['deviation'] = np.nan

            remove_index = personal[personal.movieID == i].index
#             print('Next index to remove: ' +str(remove_index))
            personal = personal.drop(remove_index)

            personal['deviation'] = np.vectorize(self.deviation)(i, personal['movieID'])
            personal['deviation_denominator'] = np.vectorize(self.get_deviation_denominator)(i, personal['movieID'])
            personal['deviation_numerator_prod'] = np.vectorize(self.numerator_part)(personal['deviation'], personal['deviation_denominator'], userID, personal['movieID'])
            
#             display(personal)
            numerator = personal['deviation_numerator_prod'].sum()
            denominator = personal['deviation_denominator'].sum()

            result = numerator / denominator
#             print('The recommentation for movieID ' +str(i)+ ' is: ' +str(result))
            
            recommend_dict[i] = result
            
            end_time_movie = time.time()
            time_movie = end_time_movie - start_time_movie
#             print('Time needed for movieID ' +str(i)+ ': ' +str(time_movie)+ ' seconds.')
        
        end_time_prediction = time.time()
        time_prediction = end_time_prediction - start_time_prediction
#         print('Time spent for the whole RECOMMENDATION: ' +str(time_prediction)+ ' seconds.')
        
        # Save similarities to use it in the future...
        # Index must be true here so that we can further read the firct column as index column:
        self.deviations.to_csv('output/calculated_deviations.dat', sep = '\t', index=True)
        self.deviations_count.to_csv('output/calculated_deviations_count.dat', sep = '\t', index=True)
        
        return recommend_dict
    
    
    
    
    
    def deviation(self, p1, p2):
        
        # Check if this is NaN value:
        if pd.isna(self.deviations.at[p1, str(p2)]) == True:
            if p1 != p2:
                dev_res = self.deviation_calculate(p1, p2)
                self.deviations.at[p1, str(p2)] = dev_res
                self.deviations.at[p2, str(p1)] = -(dev_res) # negative !
            else:
                return np.nan
        
        return self.deviations.at[p1, str(p2)]
    
    
    def deviation_calculate(self, p1, p2):
        
        data = self.X.data
        
        # Get all users that have rated p1 or p2:
        relevant_data = data[(data['movieID'] == p1) | (data['movieID'] == p2) ][['userID', 'movieID']]
        # Get only those users that have rated BOTH p1 and p2 movies:
        relevant_data = relevant_data[relevant_data.groupby(['userID'])['userID'].transform('count') == 2]
        # Get unuqie column (table) of users:
        relevant_data = relevant_data[['userID']].drop_duplicates()
        
        table = pd.DataFrame(columns = ['userID', 'movies_diff'])
        table['userID'] = relevant_data['userID']
        table = table.reset_index(drop = True)
        
        table['movies_diff'] = np.vectorize(self.calc_movies_diff)(table['userID'], p1, p2)
        
        numerator = table['movies_diff'].sum()
        denominator = table['movies_diff'].count() # so this is basically just a counter of how many sums we get
        
        self.deviations_count.at[p1, str(p2)] = denominator
        self.deviations_count.at[p2, str(p1)] = denominator # this one is positive in comparison with the deviation itself !
        
        result = float(numerator / denominator)
        return result








md_2 = MovieData('data/movies.dat')
uim_2 = UserItemData('data/user_ratedmovies.dat', min_ratings=1000)
rp_2 = SlopeOnePredictor()
rec_2 = Recommender(rp_2)
rec_2.fit(uim_2)

# In case you do not want to wait for the calculations, you can UNcomment these four lines
# and you would simply read the calculations that were done BEFORE:
# rp_2.deviations = pd.read_table('output/calculated_deviations.dat', index_col = 0)
# display(rp_2.deviations)
# rp_2.deviations_count = pd.read_table('output/calculated_deviations_count.dat', index_col = 0)
# display(rp_2.deviations_count)

print("Predictions for 78: ")
rec_items_2 = rec_2.recommend(78, n=15, rec_seen=False)
for idmovie, val in rec_items_2:
    print("Movie: {}, score: {}".format(md.get_title(idmovie), val))

Predictions for 78: 
Movie: The Usual Suspects, score: 4.325079182263173
Movie: The Lord of the Rings: The Fellowship of the Ring, score: 4.155293229840448
Movie: The Lord of the Rings: The Return of the King, score: 4.153135076202185
Movie: The Silence of the Lambs, score: 4.127978169643881
Movie: Shichinin no samurai, score: 4.119790444913598
Movie: The Lord of the Rings: The Two Towers, score: 4.083325894849594
Movie: Indiana Jones and the Last Crusade, score: 3.9670398355464194
Movie: The Incredibles, score: 3.9664496674557546
Movie: Good Will Hunting, score: 3.963362387354114
Movie: Sin City, score: 3.942619137615212
Movie: Batman Begins, score: 3.9375326640077017
Movie: A Beautiful Mind, score: 3.9140940935239508
Movie: Rain Man, score: 3.9107819079644943
Movie: Monsters, Inc., score: 3.8819375978658006
Movie: Finding Nemo, score: 3.8807711131654794
